In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from chart_studio.plotly import plot, iplot as py
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
import pandas as pd
import numpy as np
import cx_Oracle
from sqlalchemy import types, create_engine
import datetime
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from plotly.subplots import make_subplots

In [2]:
connection = cx_Oracle.connect('HCIN_ISHANDIXIT202158[AP_ds]/cR5ZKJXbL3rJ@incl02.in.prod/HWIN_USR_DEV.HOMECREDIT.IN')
curs = connection.cursor()

In [3]:
sql = f"""select TIME_CREATION_DATE PRODUCT_GOODS_TYPE,PRODUCT_CREDIT_TYPE,AMT_CREDIT, pos_gspn_v2, SCORE_APSC_NOV18,DECISION_OVERALL, RISK_AGRF30, RISK_FPD30 from ap_uwi.uw_daily_data_all
        where client_exi_3m = 'N' and TIME_CREATION_DATE >= date'2019-11-01' and TIME_CREATION_DATE < date'2020-02-01' 
        and PRODUCT_GROUP = 'CD'"""
    
data = pd.DataFrame()
size = 1000000
curs.arraysize = size
# start = time.time()
data = (data.append([i for i in curs.execute(sql)]))
# print(time.time()-start)


In [4]:
data.columns = 'TIME_CREATION_DATE,PRODUCT_CREDIT_TYPE,AMT_CREDIT,POS_GSPN_V2,SCORE,DECISION_OVERALL,RISK_AGRF30,RISK_FPD30'.split(',')
data.SCORE = data.SCORE.astype('float')

In [5]:
 data.shape

(1490300, 8)

In [6]:
# data = data.loc[data.POS_GSPN_V2.isnull()==False]
data.dropna(inplace=True)
data.shape
# data.fillna(0,inplace=True)

(1047380, 8)

In [7]:
le_key = {}
for i in ['PRODUCT_CREDIT_TYPE','POS_GSPN_V2']: 
    le = LabelEncoder()
    fit = le.fit_transform(data[i])
    data[i] = fit
    le_key[i] = pd.DataFrame(le.classes_,list(range(len(le.classes_))))

# data[['PRODUCT_GOODS_TYPE','PRODUCT_CREDIT_TYPE','POS_GSPN']]=data[['PRODUCT_GOODS_TYPE','PRODUCT_CREDIT_TYPE','POS_GSPN']].apply(lambda x:LabelEncoder().fit_transform(x))

In [8]:
test = data.loc[(data.TIME_CREATION_DATE >= datetime.datetime(2020,1,1)) & ((data.DECISION_OVERALL.isin(['Rejected','Approved'])))]
test.drop('TIME_CREATION_DATE',axis=1,inplace=True)
data = data.loc[data.TIME_CREATION_DATE < datetime.datetime(2020,1,1)]
data.drop('TIME_CREATION_DATE',axis=1,inplace=True)
rejected_data = data.loc[(data.DECISION_OVERALL == 'Rejected')]
approved_data = data.loc[(data.DECISION_OVERALL == 'Approved')]
del data
# approved_data.drop('DECISION_OVERALL',axis=1,inplace=True)

In [427]:
results = pd.DataFrame()
gini = cross_val_score(DecisionTreeClassifier(max_depth=5),approved_data[['SCORE_GM_HC2SC_OCT19']],approved_data.RISK_FPD30, cv = 3,scoring='roc_auc').mean()
for i in ['ACC_INR_GP', 
          'ACC_INR_DP', 'ACC_PCT_DP',
       'ACC_EXTRA_DP', 'AMT_CREDIT_TOTAL', 'PRODUCT_GOODS_NAME',
       'PRODUCT_PRODUCER', 'PRODUCT_CODE', 'PRODUCT_CREDIT_TYPE', 'POS_GSPN_V2',
       'POS_CITY', 'CLIENT_PCT_DI_AFTER', 'CLIENT_EMI', 'AMT_CREDIT',
       'FLAG_SAFEPAY'
         ]:
    results.loc[i,'Marginal_gini_over_score'] = (cross_val_score(DecisionTreeClassifier(max_depth=5),approved_data[['SCORE_GM_HC2SC_OCT19',i]],approved_data.RISK_FPD30, cv = 3,scoring='roc_auc').mean()/gini-1)*100
    


In [441]:
sorted_features = list(results.sort_values(by='Marginal_gini_over_score',ascending=False).index)

In [455]:
include = []
for i in  range(len(sorted_features)):
    
    include.append(sorted_features[i])
    print(include)
    gini = cross_val_score(DecisionTreeClassifier(max_depth=5),approved_data[include+['SCORE_GM_HC2SC_OCT19']],approved_data.RISK_FPD30, cv = 3,scoring='roc_auc').mean()
    if i == 0:
        print(1)
        previous_gini = gini
    else:
        print(gini/previous_gini-1)
    if (gini/previous_gini-1)<0:
        include.pop(-1)
        previous_gini = previous_gini
    else:
        previous_gini = gini   
print(include)

['POS_GSPN_V2']
1
['POS_GSPN_V2', 'AMT_CREDIT_TOTAL']
0.026294234397568284
['POS_GSPN_V2', 'AMT_CREDIT_TOTAL', 'AMT_CREDIT']
-0.00043164244357773995
['POS_GSPN_V2', 'AMT_CREDIT_TOTAL', 'ACC_INR_GP']
-0.0011121999466957577
['POS_GSPN_V2', 'AMT_CREDIT_TOTAL', 'CLIENT_EMI']
-0.003393313042815538
['POS_GSPN_V2', 'AMT_CREDIT_TOTAL', 'ACC_INR_DP']
-0.0003273995493539017
['POS_GSPN_V2', 'AMT_CREDIT_TOTAL', 'PRODUCT_CODE']
-0.0008073266586559047
['POS_GSPN_V2', 'AMT_CREDIT_TOTAL', 'PRODUCT_CREDIT_TYPE']
-1.797804754666288e-05
['POS_GSPN_V2', 'AMT_CREDIT_TOTAL', 'ACC_EXTRA_DP']
0.0001045798265313369
['POS_GSPN_V2', 'AMT_CREDIT_TOTAL', 'ACC_EXTRA_DP', 'PRODUCT_GOODS_NAME']
-0.000132328820810379
['POS_GSPN_V2', 'AMT_CREDIT_TOTAL', 'ACC_EXTRA_DP', 'ACC_PCT_DP']
-0.0011860710997281254
['POS_GSPN_V2', 'AMT_CREDIT_TOTAL', 'ACC_EXTRA_DP', 'PRODUCT_PRODUCER']
-0.0007660864235585763
['POS_GSPN_V2', 'AMT_CREDIT_TOTAL', 'ACC_EXTRA_DP', 'FLAG_SAFEPAY']
-3.7433427924282725e-05
['POS_GSPN_V2', 'AMT_CREDIT_TO

In [9]:
from scipy.stats import ks_2samp, ttest_ind

In [10]:
features = ['PRODUCT_CREDIT_TYPE','AMT_CREDIT','POS_GSPN_V2'] 
numeric = ['AMT_CREDIT']
catagorical = ['PRODUCT_CREDIT_TYPE','POS_GSPN_V2']
base_feature = features[0]
create_rgs_feature = features[1]
add_rgs_features = features[2:]


#  code for base feature, if numeric bin it
if  approved_data[features[0]].dtype == float and len(pd.unique(approved_data[features[0]]))>100:
    approved_data[features[0]] = pd.qcut(approved_data[feature],np.linspace(0,1,11))
# approved_data.ACC_EXTRA_DP = approved_data.ACC_EXTRA_DP.astype('float') 
# rejected_data.ACC_EXTRA_DP = rejected_data.ACC_EXTRA_DP.astype('float') 

In [11]:
def similar_rows(table):
    if table.columns[1] in catagorical:
        for i in range(len(table)):
            locals()['df'+str(i)] = approved_data.loc[(approved_data[table.columns[0]]==table.iloc[i,0])&(approved_data[table.columns[1]]==table.iloc[i,1])].SCORE
            locals()['pf'+str(i)] = approved_data.loc[(approved_data[table.columns[0]]==table.iloc[i,0])&(approved_data[table.columns[1]]==table.iloc[i,1])].RISK_FPD30
        for i in range(len(table)):
            for k in range(i,len(table)):
                try:
                    if table.iloc[i,0]==table.iloc[k,0]and i!=k and (ks_2samp(locals()['df'+str(i)],locals()['df'+str(k)]).pvalue) > .05 and (ttest_ind(locals()['pf'+str(i)],locals()['pf'+str(k)]).pvalue) > .05:
                        table.table_RG.iloc[k] = table.table_RG.iloc[i] 
                        print(table.columns[0]+' = '+str(table.iloc[i,0])+' '+table.columns[1]+' = '+str(table.iloc[i,1])+'   '+table.columns[0]+' = '+str(table.iloc[k,0])+' '+table.columns[1]+' = '+str(table.iloc[k,1]))
                except:
#                     print(table.iloc[i,0])
                    continue
    elif table.columns[1][:-4] in numeric:
        
        for i in range(len(table)):
            
            locals()['df'+str(i)] = approved_data.loc[(approved_data[table.columns[0]]==table.iloc[i,0])&(approved_data[table.columns[1][:-4]].map(table.iloc[i,1].__contains__))].SCORE
            locals()['pf'+str(i)] = approved_data.loc[(approved_data[table.columns[0]]==table.iloc[i,0])&(approved_data[table.columns[1][:-4]].map(table.iloc[i,1].__contains__))].RISK_FPD30
        for i in range(len(table)):
#             print(i)
            for k in range(i,len(table)):
#                 print(locals()['df'+str(k)].mean())
                try:
                    if table.iloc[i,0]==table.iloc[k,0]:
#                         print(k)
                        
                        if i!=k and (ks_2samp(locals()['df'+str(i)],locals()['df'+str(k)]).pvalue) > .05 and (ttest_ind(locals()['pf'+str(i)],locals()['pf'+str(k)]).pvalue) > .05:
                            table.table_RG.iloc[k] = table.table_RG.iloc[i] 
                            print(table.columns[0]+' = '+str(table.iloc[i,0])+' '+table.columns[1]+' = '+str(table.iloc[i,1])+'   '+table.columns[0]+' = '+str(table.iloc[k,0])+' '+table.columns[1]+' = '+str(table.iloc[k,1]))
                except:
#                     print(table.iloc[i,0])
                    continue
        
    return table

In [12]:
def rgs_make_rows(approved_data):
    
    if features[1] in catagorical:
        table = pd.pivot_table(approved_data[[features[0],features[1],'RISK_FPD30','SCORE','RISK_AGRF30']], values=['RISK_FPD30','SCORE','RISK_AGRF30'], index=[features[0],features[1]],
                            aggfunc={'RISK_FPD30': np.mean,
                                     'SCORE': np.mean,
                                     'RISK_AGRF30':'size'})
        
    else:
        for i in pd.unique(approved_data[features[0]]):
            approved_data.loc[approved_data[features[0]]==i,features[1]+'_BIN'] = pd.qcut(approved_data.loc[approved_data[features[0]]==i][features[1]],np.linspace(0,1,6),duplicates='drop')
        table = pd.pivot_table(approved_data[[features[0],features[1]+'_BIN','RISK_FPD30','SCORE','RISK_AGRF30']], values=['RISK_FPD30','SCORE','RISK_AGRF30'], index=[features[0],features[1]+'_BIN'],
                    aggfunc={'RISK_FPD30': np.mean,
                             'SCORE': np.mean,
                            'RISK_AGRF30':'size'})
    table=table.reset_index()
    table['table_RG'] = range(len(table))
    table = similar_rows(table)
    table['RISK_IN_NUM'] = table.RISK_FPD30*table.RISK_AGRF30
    table['SCORE_IN_NUM'] = table.SCORE*table.RISK_AGRF30
    old_table = table
    table = pd.DataFrame()
    table[old_table.columns[0]]=old_table.groupby('table_RG')[old_table.columns[0]].apply(lambda x: list(np.unique(x)))
    table[old_table.columns[1]]=old_table.groupby('table_RG')[old_table.columns[1]].apply(lambda x: list(np.unique(x)))
    table['CASES']=old_table.groupby('table_RG').RISK_AGRF30.sum()
    table['RISK']=old_table.groupby('table_RG').RISK_IN_NUM.sum()/table.CASES
    table['SCORE']=old_table.groupby('table_RG').SCORE_IN_NUM.sum()/table.CASES
    
    table=table.reset_index()
    new_table = table
    if table.columns[2] in catagorical:
        for i in range(len(table)):
            approved_data.loc[(approved_data[table.columns[1]].isin(table.iloc[i,1]))&(approved_data[table.columns[2]].isin(table.iloc[i,2])) ,'RG']=table.table_RG.iloc[i]
            rejected_data.loc[(rejected_data[table.columns[1]].isin(table.iloc[i,1]))&(rejected_data[table.columns[2]].isin(table.iloc[i,2])) ,'RG']=table.table_RG.iloc[i]
            test.loc[(test[table.columns[1]].isin(table.iloc[i,1]))&(test[table.columns[2]].isin(table.iloc[i,2])) ,'RG']=table.table_RG.iloc[i]
    else:
        for i in range(len(table)):
            for k in table.iloc[i,2]:
                approved_data.loc[(approved_data[table.columns[1]].isin(table.iloc[i,1]))&(approved_data[table.columns[2][:-4]].map(k.__contains__)),'RG']=table.table_RG.iloc[i]
                rejected_data.loc[(rejected_data[table.columns[1]].isin(table.iloc[i,1]))&(rejected_data[table.columns[2][:-4]].map(k.__contains__)),'RG']=table.table_RG.iloc[i]
                test.loc[(test[table.columns[1]].isin(table.iloc[i,1]))&(test[table.columns[2][:-4]].map(k.__contains__)),'RG']=table.table_RG.iloc[i]
    new_table=new_table.iloc[:,:3]
    display(table)
    iterator = 2
    for feature in features[2:]:
         
        if feature in catagorical:
            table = pd.pivot_table(approved_data[['RG',feature,'RISK_FPD30','SCORE','RISK_AGRF30']], values=['RISK_FPD30','SCORE','RISK_AGRF30'], index=['RG',feature],
                    aggfunc={'RISK_FPD30': np.mean,
                             'SCORE': np.mean,
                             'RISK_AGRF30':'count'})
        else:
            for i in pd.unique(approved_data.RG):
                approved_data.loc[approved_data.RG==i,feature+'_BIN'] = pd.qcut(approved_data.loc[approved_data.RG==i][feature],np.linspace(0,1,6),duplicates='drop')
            table = pd.pivot_table(approved_data[['RG',feature+'_BIN','RISK_FPD30','SCORE','RISK_AGRF30']], values=['RISK_FPD30','SCORE','RISK_AGRF30'], index=['RG',feature+'_BIN'],
                    aggfunc={'RISK_FPD30': np.mean,
                             'SCORE': np.mean,
                             'RISK_AGRF30':'count'})
        table=table.reset_index()
        
        table['table_RG'] = range(len(table))
        table = similar_rows(table)
        
        table = pd.merge(table, new_table, left_on='RG', right_on='table_RG',how='left')
        RG = table.RG
        table = table[table.columns[-iterator:].append(table.columns[1:6])]
        cols = table.columns[:-1]
        cols=list(cols)
        cols.append('table_RG')
        
        table.columns = cols
        table['RG'] = RG
        table['RISK_IN_NUM'] = table.RISK_FPD30*table.RISK_AGRF30
        table['SCORE_IN_NUM'] = table.SCORE*table.RISK_AGRF30
        old_table = table
        table = pd.DataFrame()
        table['RG'] =(old_table.groupby('table_RG')['RG'].apply(lambda x: list(np.unique(x)))).values
        for i in range(iterator):
            
            table[old_table.columns[i]] = (old_table.groupby('table_RG').agg({old_table.columns[i]: 'sum'}).reset_index()[old_table.columns[i]].apply(lambda a : list(set(a))))
            
        table[old_table.columns[iterator]] =(old_table.groupby('table_RG')[old_table.columns[iterator]].apply(lambda x: list(np.unique(x)))).values
        table['CASES']=old_table.groupby('table_RG').RISK_AGRF30.sum().reset_index().iloc[:,1]
        table['RISK']=(old_table.groupby('table_RG').RISK_IN_NUM.sum().reset_index().iloc[:,1]/table.CASES)
        table['SCORE']=(old_table.groupby('table_RG').SCORE_IN_NUM.sum().reset_index().iloc[:,1]/table.CASES)
        table['table_RG'] = range(len(table))
        
        display(table)
        if table.columns[iterator+1] in catagorical:
            for i in range(len(table)):
                approved_data.loc[(approved_data[table.columns[0]].isin(table.iloc[i,0]))&(approved_data[table.columns[iterator+1]].isin(table.iloc[i,iterator+1])) ,'RG_new']=table.table_RG.iloc[i]
                rejected_data.loc[(rejected_data[table.columns[0]].isin(table.iloc[i,0]))&(rejected_data[table.columns[iterator+1]].isin(table.iloc[i,iterator+1])) ,'RG_new']=table.table_RG.iloc[i]
                test.loc[(test[table.columns[0]].isin(table.iloc[i,0]))&(test[table.columns[iterator+1]].isin(table.iloc[i,iterator+1])) ,'RG_new']=table.table_RG.iloc[i]
        else:
            for i in range(len(table)):
                for k in table.iloc[i,iterator+1]:
                    approved_data.loc[(approved_data[table.columns[0]].isin(table.iloc[i,0]))&(approved_data[table.columns[iterator+1][:-4]].map(k.__contains__)),'RG_new']=table.table_RG.iloc[i]
                    rejected_data.loc[(rejected_data[table.columns[0]].isin(table.iloc[i,0]))&(rejected_data[table.columns[iterator+1][:-4]].map(k.__contains__)),'RG_new']=table.table_RG.iloc[i]
                    test.loc[(test[table.columns[0]].isin(table.iloc[i,0]))&(test[table.columns[iterator+1][:-4]].map(k.__contains__)),'RG_new']=table.table_RG.iloc[i]

        approved_data.RG = approved_data.RG_new
        approved_data.drop('RG_new',axis=1,inplace=True)
        rejected_data.RG = rejected_data.RG_new
        rejected_data.drop('RG_new',axis=1,inplace=True)
        test.RG = test.RG_new
        test.drop('RG_new',axis=1,inplace=True)
        new_table = table
        new_table=new_table.iloc[:,1:iterator+2]
        new_table.insert(0, 'table_RG', table.table_RG)
        iterator+=1
    final_table = pd.DataFrame()
    final_table= approved_data.groupby('RG').RISK_AGRF30.count().reset_index()
    final_table.columns = ['RG','CASES']
    final_table['RISK']=  approved_data.groupby('RG').RISK_FPD30.sum().reset_index().iloc[:,1]/final_table.CASES
    threshold = .01*len(approved_data)
    temp_final_table = final_table.loc[final_table.CASES>threshold]
    final_table['final'] = -999
    for i in range(len(final_table)):
        if final_table.CASES.iloc[i]<= threshold:
            final_table['final'].iloc[i] = temp_final_table.iloc[(temp_final_table.RISK-final_table.RISK.iloc[i]).abs().argsort()[:1]].RG.iloc[0] 
        else:
            final_table['final'].iloc[i] = final_table.RG.iloc[i]
    table = pd.merge(table, final_table, left_on='table_RG', right_on='RG',how='left')
    
    for i in range(len(table)):
        approved_data.loc[(approved_data[table.columns[0][:-2]]==table.table_RG.iloc[i]) ,'RG_new']=table.final.iloc[i]
        rejected_data.loc[(rejected_data[table.columns[0][:-2]]==table.table_RG.iloc[i]) ,'RG_new']=table.final.iloc[i]
        test.loc[(test[table.columns[0][:-2]]==table.table_RG.iloc[i]) ,'RG_new']=table.final.iloc[i]
    approved_data.RG = approved_data.RG_new
    approved_data.drop('RG_new',axis=1,inplace=True)
    rejected_data.RG = rejected_data.RG_new
    rejected_data.drop('RG_new',axis=1,inplace=True)
    test.RG = test.RG_new
    test.drop('RG_new',axis=1,inplace=True)
    RG = table.final
    table=table.iloc[:,1:len(features)+2]
    table['RG'] = RG
    return table

In [69]:
rgs = rgs_make_rows(approved_data)

PRODUCT_CREDIT_TYPE = 0 AMT_CREDIT_BIN = (2297.999, 10990.0]   PRODUCT_CREDIT_TYPE = 0 AMT_CREDIT_BIN = (10990.0, 11990.0]
PRODUCT_CREDIT_TYPE = 0 AMT_CREDIT_BIN = (2297.999, 10990.0]   PRODUCT_CREDIT_TYPE = 0 AMT_CREDIT_BIN = (12990.0, 15490.0]
PRODUCT_CREDIT_TYPE = 0 AMT_CREDIT_BIN = (2297.999, 10990.0]   PRODUCT_CREDIT_TYPE = 0 AMT_CREDIT_BIN = (15490.0, 99301.0]
PRODUCT_CREDIT_TYPE = 0 AMT_CREDIT_BIN = (10990.0, 11990.0]   PRODUCT_CREDIT_TYPE = 0 AMT_CREDIT_BIN = (11990.0, 12990.0]
PRODUCT_CREDIT_TYPE = 0 AMT_CREDIT_BIN = (10990.0, 11990.0]   PRODUCT_CREDIT_TYPE = 0 AMT_CREDIT_BIN = (12990.0, 15490.0]
PRODUCT_CREDIT_TYPE = 0 AMT_CREDIT_BIN = (10990.0, 11990.0]   PRODUCT_CREDIT_TYPE = 0 AMT_CREDIT_BIN = (15490.0, 99301.0]
PRODUCT_CREDIT_TYPE = 0 AMT_CREDIT_BIN = (11990.0, 12990.0]   PRODUCT_CREDIT_TYPE = 0 AMT_CREDIT_BIN = (12990.0, 15490.0]
PRODUCT_CREDIT_TYPE = 0 AMT_CREDIT_BIN = (11990.0, 12990.0]   PRODUCT_CREDIT_TYPE = 0 AMT_CREDIT_BIN = (15490.0, 99301.0]
PRODUCT_CREDIT_TYPE =

,table_RG,PRODUCT_CREDIT_TYPE,AMT_CREDIT_BIN,CASES,RISK,SCORE
0,0,[0],"[(2297.999, 10990.0], (10990.0, 11990.0], (119...",16223,0.026321,0.016057
1,5,[1],"[(1048.999, 6471.0]]",39666,0.009756,0.007982
2,6,[1],"[(6471.0, 7539.0]]",39307,0.012110,0.008965
3,7,[1],"[(7539.0, 8598.0]]",39746,0.011750,0.009713
4,8,[1],"[(8598.0, 10461.0]]",39175,0.015546,0.010674
5,9,[1],"[(10461.0, 69884.0]]",39355,0.020353,0.010694
6,10,[2],"[(1598.999, 8188.0]]",46607,0.009419,0.007811
7,11,[2],"[(8188.0, 9089.0]]",44020,0.011313,0.010847
8,12,[2],"[(9089.0, 9674.0]]",44537,0.011653,0.011470
9,13,[2],"[(9674.0, 11699.0]]",45081,0.011668,0.010405


RG = 0.0 POS_GSPN_V2 = 0   RG = 0.0 POS_GSPN_V2 = 2
RG = 0.0 POS_GSPN_V2 = 1   RG = 0.0 POS_GSPN_V2 = 2
RG = 5.0 POS_GSPN_V2 = 0   RG = 5.0 POS_GSPN_V2 = 3
RG = 5.0 POS_GSPN_V2 = 1   RG = 5.0 POS_GSPN_V2 = 3
RG = 5.0 POS_GSPN_V2 = 2   RG = 5.0 POS_GSPN_V2 = 3
RG = 8.0 POS_GSPN_V2 = 0   RG = 8.0 POS_GSPN_V2 = 3
RG = 8.0 POS_GSPN_V2 = 1   RG = 8.0 POS_GSPN_V2 = 2
RG = 8.0 POS_GSPN_V2 = 1   RG = 8.0 POS_GSPN_V2 = 3
RG = 8.0 POS_GSPN_V2 = 2   RG = 8.0 POS_GSPN_V2 = 3
RG = 9.0 POS_GSPN_V2 = 0   RG = 9.0 POS_GSPN_V2 = 3
RG = 9.0 POS_GSPN_V2 = 1   RG = 9.0 POS_GSPN_V2 = 3
RG = 9.0 POS_GSPN_V2 = 2   RG = 9.0 POS_GSPN_V2 = 3
RG = 10.0 POS_GSPN_V2 = 0   RG = 10.0 POS_GSPN_V2 = 3
RG = 10.0 POS_GSPN_V2 = 1   RG = 10.0 POS_GSPN_V2 = 3
RG = 10.0 POS_GSPN_V2 = 2   RG = 10.0 POS_GSPN_V2 = 3
RG = 11.0 POS_GSPN_V2 = 0   RG = 11.0 POS_GSPN_V2 = 3
RG = 11.0 POS_GSPN_V2 = 1   RG = 11.0 POS_GSPN_V2 = 3
RG = 11.0 POS_GSPN_V2 = 2   RG = 11.0 POS_GSPN_V2 = 3
RG = 13.0 POS_GSPN_V2 = 0   RG = 13.0 POS_GSPN_V2 = 

,RG,PRODUCT_CREDIT_TYPE,AMT_CREDIT_BIN,POS_GSPN_V2,CASES,RISK,SCORE,table_RG
0,[0.0],[0],"[(12990.0, 15490.0], (2297.999, 10990.0], (119...",[0],6369,0.022924,0.015605,0
1,[0.0],[0],"[(12990.0, 15490.0], (2297.999, 10990.0], (119...","[1, 2]",9854,0.028516,0.016349,1
2,[5.0],[1],"[(1048.999, 6471.0]]",[0],10782,0.006121,0.007526,2
3,[5.0],[1],"[(1048.999, 6471.0]]",[1],23480,0.009753,0.008044,3
4,[5.0],[1],"[(1048.999, 6471.0]]","[2, 3]",5404,0.017024,0.008622,4
5,[6.0],[1],"[(6471.0, 7539.0]]",[0],10775,0.006961,0.008672,5
6,[6.0],[1],"[(6471.0, 7539.0]]",[1],23372,0.014119,0.008992,6
7,[6.0],[1],"[(6471.0, 7539.0]]",[2],5154,0.013582,0.009449,7
8,[6.0],[1],"[(6471.0, 7539.0]]",[3],6,0.166667,0.016122,8
9,[7.0],[1],"[(7539.0, 8598.0]]",[0],11776,0.007303,0.009462,9


In [70]:
data = approved_data[['RISK_FPD30','RISK_AGRF30','SCORE','RG']].append(rejected_data[['RISK_FPD30','RISK_AGRF30','SCORE','RG']])
data['COUNT']= 1
len(data)

703153

In [71]:
grouped_df = data.groupby('RG')
df = pd.DataFrame()
for key, item in grouped_df:
    temp = grouped_df.get_group(key).sort_values(by='SCORE')
    temp['SCORE_BIN'] = pd.qcut(temp.SCORE,np.linspace(0,1,11),labels=range(10),duplicates='drop')
    df = df.append(temp)

In [72]:
opt = pd.pivot_table(df, values=['RISK_AGRF30','SCORE','COUNT'], index=['RG','SCORE_BIN'],
                    aggfunc={
#                         'RISK_FPD30': np.mean,
                             
                             'RISK_AGRF30':'sum',
                            'COUNT': 'count',
                            'SCORE': 'max',}).reset_index()
opt = pd.merge(opt,
pd.pivot_table(df.loc[df.RISK_AGRF30==1], values=['RISK_FPD30'], index=['RG','SCORE_BIN'],
                    aggfunc={
                        'RISK_FPD30': np.mean,
                    }
              ).reset_index(), on=['RG','SCORE_BIN'],how='left').sort_values(by='RISK_FPD30')

In [73]:
opt['APPROVAL']= opt.COUNT/len(df)
opt['RISK']= opt.RISK_FPD30*opt.RISK_AGRF30
opt.RISK = opt.RISK
opt.APPROVAL = opt.APPROVAL.cumsum()
# opt['OBSERVABLE'] = opt.RISK_AGRF30
opt['RISK%'] = opt.RISK/opt.RISK_AGRF30
opt['CUM_RISK'] = opt.RISK.cumsum()/opt.COUNT.cumsum()
opt['REJECT_INCLUDE_RISK'] = (opt.COUNT*opt['RISK%'])
opt['CUM_RISK_REJECT_INCLUDE'] = opt.REJECT_INCLUDE_RISK.cumsum()/opt.COUNT.cumsum()
opt

,RG,SCORE_BIN,COUNT,RISK_AGRF30,SCORE,RISK_FPD30,APPROVAL,RISK,RISK%,CUM_RISK,REJECT_INCLUDE_RISK,CUM_RISK_REJECT_INCLUDE
80,9.0,0,1289,1204,0.003842,0.000000,0.001833,0.0,0.000000,0.000000,0.000000,0.000000
152,18.0,2,1334,1217,0.004604,0.000822,0.003731,1.0,0.000822,0.000381,1.096138,0.000418
20,2.0,0,1153,1090,0.002761,0.000917,0.005370,1.0,0.000917,0.000530,1.057798,0.000570
170,20.0,0,762,678,0.003623,0.001475,0.006454,1.0,0.001475,0.000661,1.123894,0.000722
51,5.0,1,1165,1100,0.004522,0.001818,0.008111,2.0,0.001818,0.000877,2.118182,0.000946
211,24.0,1,1754,1596,0.006739,0.001880,0.010606,3.0,0.001880,0.001073,3.296992,0.001166
210,24.0,0,1755,1595,0.005120,0.001881,0.013102,3.0,0.001881,0.001194,3.300940,0.001302
30,3.0,0,2896,2702,0.003421,0.002221,0.017221,6.0,0.002221,0.001404,6.430792,0.001522
181,21.0,1,1415,1274,0.006246,0.002355,0.019233,3.0,0.002355,0.001479,3.332025,0.001609
151,18.0,1,1334,1200,0.003693,0.002500,0.021130,3.0,0.002500,0.001548,3.335000,0.001689


In [74]:
approval_rate = .6
cut_off = opt.loc[opt.APPROVAL<.6].groupby('RG').SCORE.max().reset_index()
cut_off.columns = ['RG', 'CUT_OFF']

In [112]:
SCORE = 'SCORE'
len_col_list = len(rgs.columns)-2
rgs_to_sql = rgs.groupby('RG')[rgs.columns[:len_col_list]].agg('sum')
rgs_to_sql = rgs_to_sql.reset_index()
for x in rgs_to_sql.columns[1:]:
    rgs_to_sql[x] = rgs_to_sql[x].apply(lambda x: list(set(x)))
rgs_to_sql

,RG,PRODUCT_CREDIT_TYPE,AMT_CREDIT_BIN,POS_GSPN_V2
0,0.0,[0],"[(12990.0, 15490.0], (2297.999, 10990.0], (119...",[0]
1,1.0,"[0, 1]","[(12990.0, 15490.0], (2297.999, 10990.0], (104...","[1, 2, 3]"
2,2.0,[1],"[(1048.999, 6471.0]]",[0]
3,3.0,[1],"[(1048.999, 6471.0]]",[1]
4,4.0,[1],"[(1048.999, 6471.0]]","[2, 3]"
5,5.0,[1],"[(6471.0, 7539.0]]",[0]
6,6.0,"[1, 2]","[(9674.0, 11699.0], (6471.0, 7539.0]]","[1, 2, 3]"
7,7.0,[1],"[(6471.0, 7539.0]]",[2]
8,9.0,[1],"[(7539.0, 8598.0]]",[0]
9,10.0,[1],"[(7539.0, 8598.0]]",[1]


In [113]:

sql = ''
for i in range(len(rgs_to_sql)):
    string = 'case when '
    for k in range(len_col_list+1):
        if rgs_to_sql.columns[k] in catagorical:
            for cats in rgs_to_sql[rgs_to_sql.columns[k]].iloc[i]:
                if type(le_key[rgs_to_sql.columns[k]].iloc[cats,0]) == str:
                    
                    string += f"{rgs_to_sql.columns[k]} == '{le_key[rgs_to_sql.columns[k]].iloc[cats,0]}' and "
                else:
                    string += f"{rgs_to_sql.columns[k]} == {le_key[rgs_to_sql.columns[k]].iloc[cats,0]} and "
        if rgs_to_sql.columns[k][:-4] in numeric:
            for bins in rgs_to_sql[rgs_to_sql.columns[k]].iloc[i]:
                string += f"{rgs_to_sql.columns[k][:-4]} > {bins.left} and {rgs_to_sql.columns[k][:-4]} <= {bins.right} and "
    string = string[:-4] 
    sql += string+ f" and {SCORE} <= {cut_off.CUT_OFF.iloc[i]} then 'approve'" +'\n' 
    sql += 'case when '+string+ f" and {SCORE} > {cut_off.CUT_OFF.iloc[i]} then 'reject'" +'\n'
sql += 'end REVISED_DECISION'
print(sql)
# le_key['PRODUCT_CREDIT_TYPE'].iloc[0,0]

case when PRODUCT_CREDIT_TYPE == '2X0' and AMT_CREDIT > 12990.0 and AMT_CREDIT <= 15490.0 and AMT_CREDIT > 2297.999 and AMT_CREDIT <= 10990.0 and AMT_CREDIT > 11990.0 and AMT_CREDIT <= 12990.0 and AMT_CREDIT > 15490.0 and AMT_CREDIT <= 99301.0 and AMT_CREDIT > 10990.0 and AMT_CREDIT <= 11990.0 and POS_GSPN_V2 == 1.0  and SCORE <= 0.20426111126184485 then 'approve'
case when case when PRODUCT_CREDIT_TYPE == '2X0' and AMT_CREDIT > 12990.0 and AMT_CREDIT <= 15490.0 and AMT_CREDIT > 2297.999 and AMT_CREDIT <= 10990.0 and AMT_CREDIT > 11990.0 and AMT_CREDIT <= 12990.0 and AMT_CREDIT > 15490.0 and AMT_CREDIT <= 99301.0 and AMT_CREDIT > 10990.0 and AMT_CREDIT <= 11990.0 and POS_GSPN_V2 == 1.0  and SCORE > 0.20426111126184485 then 'reject'
case when PRODUCT_CREDIT_TYPE == '2X0' and PRODUCT_CREDIT_TYPE == 'STANDARD' and AMT_CREDIT > 12990.0 and AMT_CREDIT <= 15490.0 and AMT_CREDIT > 2297.999 and AMT_CREDIT <= 10990.0 and AMT_CREDIT > 10461.0 and AMT_CREDIT <= 69884.0 and AMT_CREDIT > 6471.0 and

In [19]:
opt.loc[opt.APPROVAL<.6]

,RG,SCORE_BIN,COUNT,RISK_AGRF30,SCORE,RISK_FPD30,APPROVAL,RISK,RISK%,CUM_RISK,REJECT_INCLUDE_RISK,CUM_RISK_REJECT_INCLUDE
80,9.0,0,1289,1204,0.003842,0.000000,0.001833,0.0,0.000000,0.000000,0.000000,0.000000
152,18.0,2,1334,1217,0.004604,0.000822,0.003731,1.0,0.000822,0.000381,1.096138,0.000418
20,2.0,0,1153,1090,0.002761,0.000917,0.005370,1.0,0.000917,0.000530,1.057798,0.000570
170,20.0,0,762,678,0.003623,0.001475,0.006454,1.0,0.001475,0.000661,1.123894,0.000722
51,5.0,1,1165,1100,0.004522,0.001818,0.008111,2.0,0.001818,0.000877,2.118182,0.000946
211,24.0,1,1754,1596,0.006739,0.001880,0.010606,3.0,0.001880,0.001073,3.296992,0.001166
210,24.0,0,1755,1595,0.005120,0.001881,0.013102,3.0,0.001881,0.001194,3.300940,0.001302
30,3.0,0,2896,2702,0.003421,0.002221,0.017221,6.0,0.002221,0.001404,6.430792,0.001522
181,21.0,1,1415,1274,0.006246,0.002355,0.019233,3.0,0.002355,0.001479,3.332025,0.001609
151,18.0,1,1334,1200,0.003693,0.002500,0.021130,3.0,0.002500,0.001548,3.335000,0.001689


In [20]:
for i in pd.unique(cut_off.RG):
    RG = data.loc[data.RG==i].sort_values(by='SCORE_GM_HC2SC_OCT19')
    RG['AR'] = RG.COUNT.cumsum()/len(RG)
    RG['RISK'] = RG.RISK_FPD30.cumsum()/RG.RISK_AGRF30.cumsum()
    RG['RISK_LEFT'] =  (RG.RISK_FPD30[::-1].cumsum()/RG.RISK_AGRF30[::-1].cumsum())
    RG.RISK_LEFT = RG.RISK_LEFT.shift(1).fillna(method = 'bfill')

    x = cut_off.loc[cut_off.RG ==i].SCORE_GM_HC2SC_OCT19.iloc[0]

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(go.Scatter(x=RG.SCORE_GM_HC2SC_OCT19, y=RG.AR,
                        mode='lines',
                        name='AR'),secondary_y=False)
    fig.add_trace(
        go.Scatter(x=RG.SCORE_GM_HC2SC_OCT19, y=RG.RISK,
                        mode='lines',
                        name='RISK',
                  ),secondary_y=True,
    )
    fig.add_trace(
        go.Scatter(x=RG.SCORE_GM_HC2SC_OCT19, y=RG.RISK_LEFT,
                        mode='lines',
                        name='RISK_LEFT',
                  ),secondary_y=True,
    )

    fig.add_shape(
            # Line Vertical
            dict(
                type="line",
                x0=x,
                y0=0,
                x1=x,
                y1=1,
                line=dict(
                    color="Black",
                    width=3
                )))
    # Add figure title
    fig.update_layout(
        title_text="Cutoff ID "+str(i),xaxis_showgrid=False, yaxis_showgrid=True, yaxis2_showgrid=False, 
    )

    # Set x-axis title
    fig.update_xaxes(title_text="<b>SCORE</b>")

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>AR</b> ", secondary_y=False)
    fig.update_yaxes(title_text="<b>RISK</b> ", secondary_y=True)

    fig.show()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\ishan.dixit202158\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-20-4b9a0a93fa77>", line 2, in <module>
    RG = data.loc[data.RG==i].sort_values(by='SCORE_GM_HC2SC_OCT19')
  File "C:\Users\ishan.dixit202158\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py", line 4719, in sort_values
    k = self._get_label_or_level_values(by, axis=axis)
  File "C:\Users\ishan.dixit202158\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py", line 1706, in _get_label_or_level_values
    raise KeyError(key)
KeyError: 'SCORE_GM_HC2SC_OCT19'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\ishan.dixit202158\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 18

KeyError: 'SCORE_GM_HC2SC_OCT19'

In [21]:
test.dropna(inplace=True)

In [24]:
test = pd.merge(test,cut_off,how='left',on='RG')

In [27]:
test['NEW_RG_DECISION'] = test.apply(lambda x: 'Approved' if x.CUT_OFF > x.SCORE else 'Rejected', axis=1)

AttributeError: ("'Series' object has no attribute 'CUT_OFF'", 'occurred at index 0')

In [201]:
grouped_df = test.loc[test.NEW_RG_DECISION=='Approved'].groupby('RG')
df = pd.DataFrame()
for key, item in grouped_df:
    temp = grouped_df.get_group(key).sort_values(by='SCORE')
    temp['SCORE_BIN'] = pd.qcut(temp.SCORE,np.linspace(0,1,11),labels=range(10),duplicates='drop')
    df = df.append(temp)

In [202]:
df['COUNT']=1

In [203]:
opt = pd.pivot_table(df, values=['RISK_AGRF30','SCORE','COUNT'], index=['RG','SCORE_BIN'],
                    aggfunc={
#                         'RISK_FPD30': np.mean,
                             
                             'RISK_AGRF30':'sum',
                            'COUNT': 'count',
                            'SCORE': 'max',}).reset_index()
opt = pd.merge(opt,
pd.pivot_table(df.loc[df.RISK_AGRF30==1], values=['RISK_FPD30'], index=['RG','SCORE_BIN'],
                    aggfunc={
                        'RISK_FPD30': np.mean,
                    }
              ).reset_index(), on=['RG','SCORE_BIN'],how='left').sort_values(by='RISK_FPD30')

In [204]:
opt['APPROVAL']= opt.COUNT/len(df)
opt['RISK']= opt.RISK_FPD30*opt.RISK_AGRF30
opt.RISK = opt.RISK
opt.APPROVAL = opt.APPROVAL.cumsum()
# opt['OBSERVABLE'] = opt.RISK_AGRF30
opt['RISK%'] = opt.RISK/opt.RISK_AGRF30
opt['REJECT_INCLUDE_RISK'] = (opt.COUNT*opt['RISK%'])
opt['CUM_RISK'] = opt.REJECT_INCLUDE_RISK.cumsum()/opt.COUNT.cumsum()
opt

,RG,SCORE_BIN,COUNT,RISK_AGRF30,SCORE,RISK_FPD30,APPROVAL,RISK,RISK%,REJECT_INCLUDE_RISK,CUM_RISK
205,23.0,5,454,213,0.012939,0.000000,0.001986,0.0,0.000000,0.000000,0.000000
23,2.0,3,523,481,0.005003,0.000000,0.004273,0.0,0.000000,0.000000,0.000000
159,18.0,9,490,417,0.014783,0.000000,0.006416,0.0,0.000000,0.000000,0.000000
21,2.0,1,523,482,0.003430,0.000000,0.008703,0.0,0.000000,0.000000,0.000000
51,5.0,1,650,602,0.004328,0.000000,0.011546,0.0,0.000000,0.000000,0.000000
156,18.0,6,489,444,0.007532,0.000000,0.013685,0.0,0.000000,0.000000,0.000000
40,4.0,0,359,334,0.003788,0.000000,0.015255,0.0,0.000000,0.000000,0.000000
70,7.0,0,253,234,0.003864,0.000000,0.016362,0.0,0.000000,0.000000,0.000000
214,24.0,4,642,567,0.009492,0.000000,0.019169,0.0,0.000000,0.000000,0.000000
24,2.0,4,522,496,0.005907,0.000000,0.021452,0.0,0.000000,0.000000,0.000000


In [205]:
results = pd.DataFrame()
results = results.append(pd.DataFrame([len(test),test.loc[(test.DECISION_OVERALL=='Approved')].RISK_FPD30.mean(),opt.CUM_RISK.iloc[-1],len(test.loc[(test.DECISION_OVERALL=='Approved')])/len(test),len(test.loc[(test.NEW_RG_DECISION=='Approved')])/len(test),len(test.loc[(test.NEW_RG_DECISION=='Approved')&(test.DECISION_OVERALL=='Rejected')])]).T)
# results

In [206]:
results.columns=columns = ['CASES','RISK_V1','RISK_V2','AR_V1','AR_V2','R_V1_A_V2']

In [123]:
# results
# November 19

,CASES,RISK_V1,RISK_V2,AR_V1,AR_V2,R_V1_A_V2
0,343568.0,0.013224,0.012689,0.647552,0.696558,49525.0


In [154]:
# results
# December 18

,CASES,RISK_V1,RISK_V2,AR_V1,AR_V2,R_V1_A_V2
0,344690.0,0.01338,0.012695,0.616856,0.657611,47842.0


In [207]:
results

,CASES,RISK_V1,RISK_V2,AR_V1,AR_V2,R_V1_A_V2
0,338731.0,0.010488,0.009763,0.627973,0.675005,50667.0
